In [ ]:
# default_exp data.dataset

# Datasets
> Classes used to create `pytorch` compatible datasets.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
import torch
from torch.utils.data import Dataset
from dataclasses import dataclass
from pathlib import Path
from grade_classif.data.loaders import ImageLoader, MaskLoader, CategoryLoader
from grade_classif.data.utils import show_img, np_to_tensor
from grade_classif.data.read import get_items
from grade_classif.core import ifnone
import pandas as pd
from albumentations import Compose
import numpy as np

In [ ]:
# export
class MyDataset(Dataset):
    def __init__(self, items, labels, item_loader, label_loader):
        super().__init__()
        self.items = items
        self.labels = labels
        self.item_loader = item_loader
        self.label_loader = label_loader
        
    def __len__(self):
        return len(self.items)
    
    def __getitem__(self, i):
        item, label = self.items[i], self.labels[i]
        x = self.item_loader(item)
        y = self.label_loader(label)
        return x, y
    
    @classmethod
    def from_folder(cls, folder, label_func, item_loader, label_loader, after_open=None, recurse=True, extensions=None, include=None, exclude=None):
        folder = Path(folder)
        items, labels = get_items(folder, label_func, recurse=recurse, extensions=extensions, include=include, exclude=exclude)
        return cls(items, labels, item_loader, label_loader, after_open=after_open)
    
    def to_tensor(self, tfms=None, tfm_y=True):
        return TensorDataset(self, tfms=tfms, tfm_y=tfm_y)
    
    def split_by_list(self, train, valid, test=None):
        return SplitDataset(self.__class__(train[0], train[1], self.item_loader, self.label_loader),
                            self.__class__(valid[0], valid[1], self.item_loader, self.label_loader),
                            None if test is None else self.__class__(test[0], test[1], self.item_loader, self.label_loader))
    
    def split_by_folder(self):
        train = ([], [])
        valid = ([], [])
        test = ([], [])
        for item, label in zip(self.items, self.labels):
            if 'train' in item.parts:
                train[0].append(item)
                train[1].append(label)
            elif 'valid' in item.parts:
                valid[0].append(item)
                valid[1].append(label)
            elif 'test' in item.parts:
                test[0].append(item)
                test[1].append(label)
        if test[0] == []:
            test = None
        return self.split_by_list(train, valid, test)
    
    def split_by_csv(self, csv, column='split'):
        df = pd.read_csv(csv, header='infer')
        train = ([], [])
        valid = ([], [])
        test = ([], [])
        train_ids = df.loc[df[column] == 'train', 'scan']
        valid_ids = df.loc[df[column] == 'valid', 'scan']
        test_ids = df.loc[df[column] == 'test', 'scan']
        for item, label in zip(self.items, self.labels):
            scan_name = item.parent.name
            if scan_name in train_ids.values:
                train[0].append(item)
                train[1].append(label) 
            elif scan_name in valid_ids.values:
                valid[0].append(item)
                valid[1].append(label)
            elif scan_name in test_ids.values:
                test[0].append(item)
                test[1].append(label)
        if test[0] == []:
            test = None
        return self.split_by_list(train, valid, test)    

Base class for all datasets. It is constructed using list of `items` and `labels`, most of the time as `Path` or `str` objects, with corresponding `item_loader` and `label_loader` as `ItemLoader` objects.

In [ ]:
# export
class ClassDataset(MyDataset):
    def __init__(self, *args, n_classes=2, **kwargs):
        super().__init__(*args, **kwargs)
        self.n_classes = n_classes

In [ ]:
# export
class ImageClassifDataset(ClassDataset):
    def show(self, k, ax=None, figsize=(3,3), hide_axis=True, cmap='viridis', **kwargs):
        x, y  = self[k]
        y = self.label_loader.classes[y]
        ax = show_img(x, ax=ax, hide_axis=hide_axis, cmap=cmap, figsize=figsize, title=str(y), **kwargs)
        
    def show_rand(self, ax=None, figsize=(3,3), hide_axis=True, cmap='viridis', **kwargs):
        k = random.randint(0, len(self)-1)
        self.show(k, ax=ax, figsize=figsize, hide_axis=hide_axis, cmap=cmap, **kwargs)
        
    @classmethod
    def from_folder(cls, folder, label_func, n_classes=None, classes=None, recurse=True, extensions=None, include=None, exclude=None, **kwargs):
        folder = Path(folder)
        items, labels = get_items(folder, label_func, recurse=recurse, extensions=extensions, include=include, exclude=exclude)
        return cls(items, labels, ImageLoader(**kwargs), CategoryLoader(n_classes, classes), n_classes=ifnone(n_classes, len(classes)))

In [ ]:
# export
class ImageSegmentDataset(ClassDataset):
    def show(self, k, ax=None, figsize=(3,3), title=None, hide_axis=True, cmap='tab20', **kwargs):
        x, y  = self[k]
        ax = show_img(x, ax=ax, hide_axis=hide_axis, cmap=cmap, figsize=figsize, **kwargs)
        ax = show_img(y, ax=ax, hide_axis=hide_axis, cmap=cmap, figsize=figsize,
                        interpolation='nearest', alpha=alpha, vmin=0, title=title, **kwargs)

    def show_rand(self, ax=None, figsize=(3,3), hide_axis=True, cmap='tab20', **kwargs):
        k = random.randint(0, len(self)-1)
        self.show(k, ax=ax, figsize=figsize, hide_axis=hide_axis, cmap=cmap, **kwargs)
        
    @classmethod
    def from_folder(cls, folder, label_func, n_classes=None, classes=None, recurse=True, extensions=None, include=None, exclude=None):
        folder = Path(folder)
        items, labels = get_items(folder, label_func, recurse=recurse, extensions=extensions, include=include, exclude=exclude)
        return cls(items, labels, ImageLoader(), MaskLoader(), n_classes=ifnone(n_classes, len(classes)))

In [ ]:
# export
class NormDataset(MyDataset):
    def show(self, k, axs=None, figsize=(3,3), title=None, hide_axis=True, cmap='viridis', **kwargs):
        x, y  = self[k]
        axs = ifnone(axs, plt.subplots(1, 2, figsize=figsize)[1])
        ax = show_img(x, ax=axs[0], hide_axis=hide_axis, cmap=cmap, figsize=figsize, **kwargs)
        ax = show_img(y, ax=axs[1], hide_axis=hide_axis, cmap=cmap, figsize=figsize, **kwargs)

    def show_rand(self, axs=None, figsize=(3,3), hide_axis=True, cmap='viridis', **kwargs):
        k = random.randint(0, len(self)-1)
        self.show(k, axs=axs, figsize=figsize, hide_axis=hide_axis, cmap=cmap, **kwargs)
        
    @classmethod
    def from_folder(cls, folder, label_func, csv, recurse=True, extensions=None, include=None, exclude=None):
        df = pd.read_csv(csv)
        vals = df.loc[df['category'] == 1, 'imageId'].values
        def filt(fn):
            return fn.parent.stem in vals      
        folder = Path(folder)
        items, labels = get_items(folder, label_func, recurse=recurse, extensions=extensions, include=include, exclude=exclude, filterfunc=filt)
        return cls(items, labels, ImageLoader(open_mode='3G'), ImageLoader())

In [ ]:
# export
@dataclass
class SplitDataset:
    train: Dataset
    valid: Dataset
    test: Dataset = None
        
    def to_tensor(self, tfms=None, tfm_y=True, test_tfms=None):
        tfms = ifnone(tfms, (None, None))
        self.train = self.train.to_tensor(tfms=tfms[0], tfm_y=tfm_y)
        self.valid = self.valid.to_tensor(tfms=tfms[1], tfm_y=tfm_y)
        if self.test is not None:
            self.test = self.test.to_tensor(tfms=test_tfms, tfm_y=tfm_y)
        return self

In [ ]:
# export
class TensorDataset(Dataset):
    def __init__(self, ds, tfms=None, tfm_y=True):
        self._ds = ds
        self.tfms = ifnone(tfms, [])
        self.tfm_y = tfm_y
        
    def __len__(self):
        return len(self._ds)
    
    def __getitem__(self, i):
        x, y = self._ds[i]
        if self.tfms != []:
            aug = Compose(self.tfms)
            augmented = aug(image=x, mask=y if self.tfm_y else None)
            x = augmented['image']
            if self.tfm_y: 
                y = augmented['mask']
        x = np_to_tensor(x, type(self._ds.item_loader).__name__.lower().replace('loader', ''))
        y = np_to_tensor(y, type(self._ds.label_loader).__name__.lower().replace('loader', ''))
        return x, y
    
    def show(self, k, **kwargs):
        self._ds.show(k, **kwargs)
        
    def show_rand(self, **kwargs):
        self._ds.show_rand(**kwargs)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_train.ipynb.
Converted 02_predict.ipynb.
Converted 10_data.read.ipynb.
Converted 11_data.loaders.ipynb.
Converted 12_data.dataset.ipynb.
Converted 13_data.utils.ipynb.
Converted 14_data.transforms.ipynb.
Converted 20_models.plmodules.ipynb.
Converted 21_models.modules.ipynb.
Converted 22_models.utils.ipynb.
Converted 23_models.hooks.ipynb.
Converted 24_models.metrics.ipynb.
Converted 80_params.defaults.ipynb.
Converted 81_params.parser.ipynb.
Converted 99_index.ipynb.
